In [30]:
import os

In [31]:
path = "D:\Projects\Sem5\ECGAnalysisDashbaord\mit-bih-arrhythmia-database-1.0.0"
record_list = []

for filename in os.listdir(path):
    if filename.endswith(".atr"):  # Assuming the records end with .atr extension
        record_name = os.path.splitext(filename)[0]  # Get the filename without extension
        record_list.append(record_name)

print(record_list)

['100', '101', '102', '103', '104', '105', '106']


<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\PC PAL\AppData\Local\Temp\ipykernel_7912\2413215257.py:1: SyntaxWarning: invalid escape sequence '\P'
  path = "D:\Projects\Sem5\ECGAnalysisDashbaord\mit-bih-arrhythmia-database-1.0.0"


In [32]:
import torch
import numpy as np
import wfdb

# Load the trained model
model_path = "D:\Projects\Sem5\ECGAnalysisDashbaord\models\cnn_lstm_model.pth"

def load_model(model_path):
    model = CNN_LSTM()
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    return model

model = load_model(model_path)
model.eval()  # Set model to evaluation mode

# Dictionary to count predictions
prediction_count = {
    "0": 0,
    "1": 0
}

for option2 in record_list:
    # Load ECG signal
    data = f"D:\Projects\Sem5\ECGAnalysisDashbaord\mit-bih-arrhythmia-database-1.0.0/{option2}"
    record = wfdb.rdrecord(data, smooth_frames=True)
    ecg_signal = record.p_signal[:8000, 0]  # Assuming you want the first channel

    device = 'cpu'
    input_data = torch.tensor(ecg_signal, dtype=torch.float32).unsqueeze(0).unsqueeze(1).to(device)

    # Make predictions
    with torch.no_grad():
        output = model(input_data)

    predictions = torch.argmax(output, dim=1).cpu().numpy()[0]

    prediction_count[str(predictions)] += 1

# Display results
print("Prediction Counts:")
print(recor_name)
print(f"Predicted class 0: {prediction_count['0']} times")
print(f"Predicted class 1: {prediction_count['1']} times")


<>:6: SyntaxWarning: invalid escape sequence '\P'
<>:24: SyntaxWarning: invalid escape sequence '\P'
<>:6: SyntaxWarning: invalid escape sequence '\P'
<>:24: SyntaxWarning: invalid escape sequence '\P'
C:\Users\PC PAL\AppData\Local\Temp\ipykernel_7912\192885587.py:6: SyntaxWarning: invalid escape sequence '\P'
  model_path = "D:\Projects\Sem5\ECGAnalysisDashbaord\models\cnn_lstm_model.pth"
C:\Users\PC PAL\AppData\Local\Temp\ipykernel_7912\192885587.py:24: SyntaxWarning: invalid escape sequence '\P'
  data = f"D:\Projects\Sem5\ECGAnalysisDashbaord\mit-bih-arrhythmia-database-1.0.0/{option2}"


Prediction Counts:
[]
Predicted class 0: 1 times
Predicted class 1: 6 times


In [33]:
print(recor_name)

[]


In [34]:
# myocardial_infarction.py
import torch
import numpy as np
from torch import nn
from sklearn.preprocessing import LabelEncoder

# Define the CNN class
class CNN(nn.Module):
    def __init__(self, hidden):
        super(CNN, self).__init__()
        self.hiddenLast = hidden
        self.hidden1 = 187
        self.hidden2 = 64
        self.hidden3 = 64
        self.hidden4 = 64
        self.hidden5 = 64
        self.hidden6 = 64
        self.kernel_size = 2
        self.activation = nn.ReLU()
        self.first = True

        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=self.hidden1, kernel_size=self.kernel_size),
            nn.BatchNorm1d(num_features=self.hidden1),
            self.activation,
        )

        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels=self.hidden1, out_channels=self.hidden2, kernel_size=self.kernel_size),
            nn.BatchNorm1d(num_features=self.hidden2),
            self.activation,
            nn.MaxPool1d(kernel_size=2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels=self.hidden2, out_channels=self.hidden3, kernel_size=self.kernel_size),
            nn.BatchNorm1d(num_features=self.hidden3),
            self.activation,
            nn.MaxPool1d(kernel_size=2)
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels=self.hidden3, out_channels=self.hidden4, kernel_size=self.kernel_size),
            nn.BatchNorm1d(num_features=self.hidden4),
            self.activation,
            nn.MaxPool1d(kernel_size=2)
        )
        
        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels=self.hidden4, out_channels=self.hidden5, kernel_size=self.kernel_size),
            nn.BatchNorm1d(num_features=self.hidden5),
            self.activation,
            nn.MaxPool1d(kernel_size=2)
        )
        
        self.conv6 = nn.Sequential(
            nn.Conv1d(in_channels=self.hidden5, out_channels=self.hidden6, kernel_size=self.kernel_size),
            nn.BatchNorm1d(num_features=self.hidden6),
            self.activation,
            nn.MaxPool1d(kernel_size=2)
        )
        
        self.convLast = nn.Sequential(
            nn.Conv1d(in_channels=self.hidden4, out_channels=self.hiddenLast, kernel_size=self.kernel_size),
            nn.BatchNorm1d(num_features=self.hiddenLast),
            self.activation,
            nn.Dropout(0.2),
            nn.MaxPool1d(kernel_size=2)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.convLast(x)
        return x

# Define the CNN_LSTM class
class CNN_LSTM(nn.Module):
    def __init__(self):
        super(CNN_LSTM, self).__init__()
        self.cnn_output = 128
        self.seq_length_after_cnn = calculate_output_length([
            CNN(hidden=128).conv1[0],
            CNN(hidden=128).conv2[0],
            CNN(hidden=128).conv2[3],
            CNN(hidden=128).conv3[0],
            CNN(hidden=128).conv3[3],
            CNN(hidden=128).conv4[0],
            CNN(hidden=128).conv4[3],
            CNN(hidden=128).conv5[0],
            CNN(hidden=128).conv5[3],
            CNN(hidden=128).conv6[0],
            CNN(hidden=128).conv6[3],
            CNN(hidden=128).convLast[0],
            CNN(hidden=128).convLast[4]
        ], 187)
        
        self.hidden_lstm = 128
        self.cnn = CNN(hidden=self.cnn_output)
        self.lstm = nn.LSTM(input_size=self.cnn_output, hidden_size=self.hidden_lstm, batch_first=True, num_layers=3)
        self.fc = nn.Linear(in_features=self.hidden_lstm * self.seq_length_after_cnn, out_features=2)

    def forward(self, x):
        x = self.cnn(x)  
        x = x.permute(0, 2, 1)  
        x, _ = self.lstm(x)  
        x = x.reshape(-1, self.hidden_lstm * self.seq_length_after_cnn)
        x = self.fc(x)
        return x

def calculate_output_length(layers, initial_length):
    length = initial_length
    for layer in layers:
        if isinstance(layer, nn.Conv1d):
            kernel_size = layer.kernel_size[0]
            stride = layer.stride[0]
            padding = layer.padding[0]
            dilation = layer.dilation[0]
            length = (length + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1
        elif isinstance(layer, nn.MaxPool1d):
            kernel_size = layer.kernel_size
            stride = layer.stride
            length = (length - kernel_size) // stride + 1
    return length

def preprocess_signal(signal):
    signal = np.expand_dims(signal, axis=0)  # Add channel dimension
    signal = torch.tensor(signal, dtype=torch.float32)
    return signal

def predict(model, signal, encoder):
    signal = preprocess_signal(signal)
    output = model(signal)
    pred = output.argmax(dim=1).item()
    return encoder.inverse_transform([pred])[0]

def load_model(model_path):
    model = CNN_LSTM()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Label encoder
encoder = LabelEncoder()
encoder.classes_ = np.array(['Normal', 'Abnormal'])



In [35]:
import torch
import numpy as np
import wfdb

# Load the trained model
model_path = r"D:\Projects\Sem5\ECGAnalysisDashbaord\models\cnn_lstm_model.pth"

def load_model(model_path):
    model = CNN_LSTM()  # Assuming CNN_LSTM is your model class
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    return model

model = load_model(model_path)
model.eval()  # Set model to evaluation mode

# Assuming record_list is already populated with record names

# Dictionary to count predictions
prediction_count = {
    "0": 0,
    "1": 0
}

for record_name in record_list:
    # Load ECG signal
    data_path = rf"D:\Projects\Sem5\ECGAnalysisDashbaord\mit-bih-arrhythmia-database-1.0.0\{record_name}"
    record = wfdb.rdrecord(data_path, smooth_frames=True)
    ecg_signal = record.p_signal[:8000, 0]  # Assuming you want the first channel

    device = 'cpu'
    input_data = torch.tensor(ecg_signal, dtype=torch.float32).unsqueeze(0).unsqueeze(1).to(device)

    # Make predictions
    with torch.no_grad():
        output = model(input_data)

    predictions = torch.argmax(output, dim=1).cpu().numpy()[0]

    prediction_count[str(predictions)] += 1

    # Print prediction for each record
    print(f"Record: {record_name}, Predicted class: {predictions}")

# Display overall prediction counts
print("\nPrediction Counts:")
print(f"Predicted class 0: {prediction_count['0']} times")
print(f"Predicted class 1: {prediction_count['1']} times")


Record: 100, Predicted class: 1
Record: 101, Predicted class: 1
Record: 102, Predicted class: 1
Record: 103, Predicted class: 1
Record: 104, Predicted class: 0
Record: 105, Predicted class: 1
Record: 106, Predicted class: 1

Prediction Counts:
Predicted class 0: 1 times
Predicted class 1: 6 times
